In [ ]:
## case t > p

In [ ]:
import sys
import subprocess
from pathlib import Path

REPO_ROOT = Path("/Users/aqibsyed/Documents/Research/Clustering/AP code implementations")
SCRIPT = REPO_ROOT / "scripts" / "run_synthetic_mcss.py"
PYTHON = sys.executable  # keeps us in the same environment as the notebook kernel

In [ ]:
# Cell B – identity covariance, p=100, t=1k
subprocess.run(
    [
        PYTHON,
        str(SCRIPT),
        "--scenario",
        "tgt_gt_iso_p100_t1000",
        "--b",
        "50",
        "--overwrite-mcss",
    ],
    check=True,
)

Available scenarios:
    tgt_gt_iso_p100_t10000  (t>p, identity_var1)
     tgt_gt_iso_p100_t5000  (t>p, identity_var1)
     tgt_gt_iso_p100_t1000  (t>p, identity_var1)
    tgt_gt_iso_p200_t10000  (t>p, identity_var1)
     tgt_gt_iso_p200_t5000  (t>p, identity_var1)
     tgt_gt_iso_p200_t1000  (t>p, identity_var1)
    tgt_gt_iso_p400_t10000  (t>p, identity_var1)
     tgt_gt_iso_p400_t5000  (t>p, identity_var1)
     tgt_gt_iso_p400_t1000  (t>p, identity_var1)
  tgt_gt_var10_p100_t10000  (t>p, identity_var10)
   tgt_gt_var10_p100_t5000  (t>p, identity_var10)
   tgt_gt_var10_p100_t1000  (t>p, identity_var10)
  tgt_gt_var10_p200_t10000  (t>p, identity_var10)
   tgt_gt_var10_p200_t5000  (t>p, identity_var10)
   tgt_gt_var10_p200_t1000  (t>p, identity_var10)
  tgt_gt_var10_p400_t10000  (t>p, identity_var10)
   tgt_gt_var10_p400_t5000  (t>p, identity_var10)
   tgt_gt_var10_p400_t1000  (t>p, identity_var10)
  tgt_gt_toeplitz_p100_t10000  (t>p, toeplitz)
  tgt_gt_toeplitz_p100_t5000  (t>p, toepl

CompletedProcess(args=['/opt/anaconda3/bin/python', '/Users/aqibsyed/Documents/Research/Clustering/AP code implementations/scripts/run_synthetic_mcss.py', '--list'], returncode=0)

In [ ]:
# Cell 3 – run every t > p case (identity, Toeplitz, high-variance; b=200; damping 0.50 & 0.85)
subprocess.run([PYTHON, str(SCRIPT), "--regime", "t>p"], check=True)

In [ ]:
# Cell B – identity covariance, p=100, t=1k
subprocess.run(
    [
        PYTHON,
        str(SCRIPT),
        "--scenario",
        "tgt_gt_iso_p100_t1000",
        "--b",
        "50",
        "--overwrite-mcss",
    ],
    check=True,
)

In [ ]:
# Cell C – Toeplitz covariance, p=100, t=1k
subprocess.run(
    [
        PYTHON,
        str(SCRIPT),
        "--scenario",
        "tgt_gt_toeplitz_p100_t1000",
        "--b",
        "50",
        "--overwrite-mcss",
    ],
    check=True,
)

In [ ]:
# Cell D – high-variance diagonal, p=100, t=1k
subprocess.run(
    [
        PYTHON,
        str(SCRIPT),
        "--scenario",
        "tgt_gt_var10_p100_t1000",
        "--b",
        "50",
        "--overwrite-mcss",
    ],
    check=True,
)

## Initial runs with Siamak's data

In [1]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd

REPO_ROOT = Path("..").resolve()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

from apclust import run_mcss_ap

DATA_ROOT = REPO_ROOT / "Data"
synthetic_path = DATA_ROOT / "samples_original_1_for_GaussMix4_DiagCovar_highVar_2Kdim_n1K.csv"
synthetic = pd.read_csv(synthetic_path, header=None)
X = synthetic.to_numpy(dtype=np.float64)

OUTPUT_BASE = REPO_ROOT / "results" / "mcss" / "synthetic_gaussmix4_highvar"
OUTPUT_BASE.mkdir(parents=True, exist_ok=True)

for damping in (0.50, 0.85):
    run_mcss_ap(
        X,
        dataset_name=f"damping{damping:.2f}",
        out_dir=OUTPUT_BASE,
        b=200,
        train_frac=0.80,
        random_seed=0,
        ap_params={
            "damping": damping,
            "max_iter": 400,
            "convergence_iter": 50,
            "random_state": 0,
            "affinity": "euclidean",
        },
        metrics=("ari", "ami"),
    )

/opt/anaconda3/lib/python3.12/site-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/cluster/_affinity_propagation.py:140: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [2]:
import pandas as pd

for damping in (0.50, 0.85):
    summary = pd.read_csv(OUTPUT_BASE / f"damping{damping:.2f}" / "mcss_summary.csv")
    mean_k = summary["train_n_clusters"].mean()
    var_k = summary["train_n_clusters"].var()
    mode_k = summary["train_n_clusters"].mode()[0]
    mode_pct = (summary["train_n_clusters"] == mode_k).mean() * 100
    conv_pct = summary["train_converged"].mean() * 100

    print(f"damping={damping:.2f}")
    print(f"  E[K]      : {mean_k:.3f}")
    print(f"  Var[K]    : {var_k:.3f}")
    print(f"  Mode[K]   : {mode_k} ({mode_pct:.1f} %)")
    print(f"  %Converged: {conv_pct:.1f} %\n")

damping=0.50
  E[K]      : 14.095
  Var[K]    : 0.840
  Mode[K]   : 14 (46.0 %)
  %Converged: 99.0 %

damping=0.85
  E[K]      : 14.140
  Var[K]    : 0.935
  Mode[K]   : 14 (46.0 %)
  %Converged: 100.0 %

